# Tratamento dos arquivos de treinamento e teste

Nesse notebook trataremos os arquivos `trasact_train.csv`, `transact_class.csv` e `realclass_t1.csv`.

O objetivo é preparar os dados na forma de CSV para utiliza-los na fase seguinte, escolha de paramêtros dos algoritmo de classificação.

Ao final do processo serão gerados 3 pares de arquivos, cada par referente a uma estratégia para tratamento de missing values.

### Importar as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np

### Ler os arquivos csv

In [2]:
transact_train = pd.read_csv('data/transact_train.csv', sep='|', na_values='?')
transact_class = pd.read_csv('data/transact_class.csv', sep='|', na_values='?')
realclass_t1 = pd.read_csv('data/realclass_t1.csv', sep='|', na_values='?')

# Remover os dados de ID do usuário
transact_train.drop('customerNo', axis=1, inplace=True)
transact_class.drop('customerNo', axis=1, inplace=True)

### Diminuir a granularidade dos dados

In [3]:
def reduceGranularity(data):
    ant = data['sessionNo'][0]
    indexes = []
    reduced_data = pd.DataFrame()
    for index, row in data.iterrows():
        if row['sessionNo'] != ant:
            indexes.append(index)
        ant = row['sessionNo']
    for index in range(0, len(indexes)):
        indexes[index] -= 1
    indexes.append(len(data) - 1)
    reduced_data = data.iloc[indexes].set_index('sessionNo')
    return reduced_data

In [4]:
training = reduceGranularity(transact_train)
testing = reduceGranularity(transact_class)

### Juntar `realclass_t1` com `testing`

In [5]:
testing = testing.join(realclass_t1.set_index('sessionNo')).rename(columns={'prediction': 'order'})

### Substituír valores nominais por valores numéricos

In [6]:
# Mapeando os valores nominais
def mappingValues(df):
    if(df['order'].dtype == 'int64'):
        pass
    else:
        df['order'] = df['order'].map({'y': 1, 'n': 0})
    df['onlineStatus'] = df['onlineStatus'].map({'y': 1, 'n': 0})
    df['availability'] = df['availability'].map({
        'completely orderable': 1, 'completely not orderable': 2,'mainly orderable': 3,
        'mixed': 4,'completely not determinable': 5, 'mainly not orderable': 6, 'mainly not determinable': 7
    })
    
mappingValues(training)
mappingValues(testing)

### Substituir missing values

In [7]:
# Substitui valores pela moda
def fillNaByMode(data):
    new_data = data.copy()
    for col in new_data:
        new_data[col].fillna(value=new_data[col].mode().iloc[0], inplace=True)
    return new_data
 
# Substitui valores inteiros pela média e valores nominais pela moda
def fillNaByMean(data):
    new_data = data.copy()
    for col in new_data:
        new_data[col].fillna(value=new_data[col].mean(), inplace=True)
    return new_data

In [8]:
# Treinamento
training_mean = fillNaByMean(training)
training_mode = fillNaByMode(training)
training_no_na = training.dropna()

# Teste
testing_mean = fillNaByMean(testing)
testing_mode = fillNaByMode(testing)
testing_no_na = testing.dropna()

### Normalizar os dados utilizando teorema de Tales

In [9]:
def normalize(df):
    return (df - df.min())/(df.max() - df.min())

training_mean = normalize(training_mean)
training_mode = normalize(training_mode)
training_no_na = normalize(training_no_na)

testing_mean = normalize(training_mean)
testing_mode = normalize(training_mode)
testing_no_na = normalize(training_no_na)

### Salvar os dados em arquivos CSV

In [10]:
# Treinamento
training_mean.to_csv('data/training_mean.csv')
training_mode.to_csv('data/training_mode.csv')
training_no_na.to_csv('data/training_no_na.csv')

# Teste
testing_mean.to_csv('data/testing_mean.csv')
testing_mode.to_csv('data/testing_mode.csv')
testing_no_na.to_csv('data/testing_no_na.csv')